In [83]:
import pandas as pd
import numpy as np

In [84]:
# http://www.cde.ca.gov/ta/ac/sa/accessdata1516.asp

# http://www3.cde.ca.gov/researchfiles/sarc/sarc1516/schdetail.txt
publicschools = pd.read_table("publicschools.txt") 
publicschools.head()

# http://www.cde.ca.gov/ds/si/ps/
privateschools = pd.read_csv("privateschools1516.csv")
privateschools.head()

,Affidavit ID,County,CDS Code,School,Street,City,State,Zip,Mailing Street,Mailing City,...,Religious Denomination,Director/Principal Officer Salutation,Director/Principal Officer First Name,Director/Principal Officer Last Name,Director/Principal Officer Title,Director/Principal Officer Email,Custodian of Records Salutation,Custodian of Records First Name,Custodian of Records Last Name,Custodian of Records Email
0,171089,Alameda,01612596910343,Agnes Memorial Christian Academey,2372 International Boulevard,Oakland,CA,94601,NaN,NaN,...,NaN,Rev.,L,Simmons,Superintendent,NaN,Rev.,M,Simmons,NaN
1,175255,Alameda,01611926965909,All Saints Catholic School,22870 Second Street,Hayward,CA,94541,NaN,NaN,...,Roman Catholic,Mrs.,Jennifer,Diaz,Principal,jdiaz@csdo.org,Ms.,Susan,Porter,sporter@csdo.org
2,168786,Alameda,01611767103799,Alsion Montessori Middle/High,750 Witherly Lane,Fremont,CA,94539,NaN,NaN,...,NaN,Mrs.,Rosemary,Petersen,Head of School,alsion.rosemary@comcast.net,Mrs.,Rosemary,Petersen,alsion.rosemary@comcast.net
3,171033,Alameda,01611436135743,American International Montessori School,3339 Martin Luther King Jr. Way,Berkeley,CA,94703,NaN,NaN,...,NaN,Mr.,Ernest,Mahr,Director,office@AIMmontessori.com,Mr.,Ernest,Mahr,office@AIMmontessori.com
4,175156,Alameda,01611506133441,"Anchor Education, Inc. dba Sterling West -- Sa...",19234 Lake Chabot Road,Castro Valley,CA,94546,PO Box 368,San Leandro,...,NaN,Mr.,Guy,Fussell,Executive Director,guy.fussell@sterlinged.org,Mrs.,Jacqueline,Holt,jacqueline.holt@sterlinged.org


In [85]:
# Pasadena Public & Private Schools

listofCDS = privateschools[privateschools["City"] == "Pasadena"]["CDS Code"].values
listofCDS = np.append(listofCDS, publicschools[publicschools["CITY"] == "Pasadena"]["CDSCode"].values)
listofCDS = listofCDS.astype('int')
len(listofCDS)

63

In [86]:
pasadena_publics = publicschools[publicschools["CITY"] == "Pasadena"][["CDSCode", "School"]]
pasadena_privates = pd.DataFrame(privateschools[privateschools["City"] == "Pasadena"][["CDS Code", "School "]])
pasadena_privates.columns = ["CDSCode", "School"]

pasadena_schools = pd.concat([pasadena_publics, pasadena_privates])
pasadena_schools.head()

,CDSCode,School
2682,19648811931062,Blair High
2683,19648811931674,Marshall Fundamental
2684,19648811932409,Norma Coombs Elementary
2685,19648811936103,John Muir High
2686,19648811936806,Rose City High (Continuation)
2687,19648811936822,Pasadena High
2689,19648816021547,Cleveland Elementary
2690,19648816021554,Don Benito Fundamental
2691,19648816021570,Field (Eugene) Elementary
2693,19648816021612,Hamilton Elementary


In [104]:
# http://www.cde.ca.gov/ta/ac/sa/accessdata1516.asp
# CAASPP Results 15-16
caaspp = pd.read_csv("CAASPPTestResults.csv")
pasadena_caaspp = caaspp[caaspp["CDS"].isin(pasadena_schools.CDSCode.values)]
pasadena_caaspp = pd.merge(pasadena_caaspp, pasadena_schools, how='left', left_on="CDS", right_on="CDSCode")
pasadena_caaspp.head() ## why does it only return public schools ughhh

,CDS,County Code,District Code,School Code,SARCYear,SELA_Y1,SELA_Y2,DELA_Y1,DELA_Y2,STELA_Y1,STELA_Y2,SMATH_Y1,SMATH_Y2,DMATH_Y1,DMATH_Y2,STMATH_Y1,STMATH_Y2,CDSCode,School
0,19648816021687,19,64881,6021687,2015-16,14,21,37,42,44,48,11,9,29,30,34,36,19648816021687,Madison Elementary
1,19648816021638,19,64881,6021638,2015-16,19,27,37,42,44,48,19,18,29,30,34,36,19648816021638,Jefferson Elementary
2,19648811936806,19,64881,1936806,2015-16,5,NaN,37,42,44,48,0,NaN,29,30,34,36,19648811936806,Rose City High (Continuation)
3,19648816021752,19,64881,6021752,2015-16,20,20,37,42,44,48,10,10,29,30,34,36,19648816021752,Washington Middle
4,19648816021612,19,64881,6021612,2015-16,63,64,37,42,44,48,57,51,29,30,34,36,19648816021612,Hamilton Elementary


In [106]:
# Pasadena has so many! Let's compare it to counts for public schools.
privateschools_enr_cities = privateschools.groupby("City").sum()[["Total Enrollment"]].reset_index()
privateschools_enr_cities.sort_values("Total Enrollment", ascending=False)[:10]

,City,Total Enrollment
272,Los Angeles,37559
445,San Francisco,23964
448,San Jose,20526
442,San Diego,16040
433,Sacramento,12568
374,Pasadena,8158
346,Oakland,7212
462,Santa Ana,5046
349,Ojai,4770
422,Riverside,4676


In [90]:
enr15 = pd.read_table("publicschools15enr.txt")
enr15 = enr15.groupby("CDS_CODE").sum()[["ENR_TOTAL"]].reset_index()
pasadena_publics_enr = pd.merge(pasadena_publics, enr15, how='left', left_on="CDSCode", right_on="CDS_CODE")
pasadena_publics_enr

,CDSCode,School,CDS_CODE,ENR_TOTAL
0,19648811931062,Blair High,19648811931062,962
1,19648811931674,Marshall Fundamental,19648811931674,1971
2,19648811932409,Norma Coombs Elementary,19648811932409,458
3,19648811936103,John Muir High,19648811936103,902
4,19648811936806,Rose City High (Continuation),19648811936806,265
5,19648811936822,Pasadena High,19648811936822,1871
6,19648816021547,Cleveland Elementary,19648816021547,193
7,19648816021554,Don Benito Fundamental,19648816021554,608
8,19648816021570,Field (Eugene) Elementary,19648816021570,502
9,19648816021612,Hamilton Elementary,19648816021612,590


In [101]:
publicschools_enr = pd.merge(publicschools, enr15, how='left', left_on="CDSCode", right_on="CDS_CODE")
publicschools_enr_cities = publicschools_enr.groupby("CITY").sum()[["ENR_TOTAL"]].reset_index()
publicschools_enr_cities.sort_values("ENR_TOTAL", ascending=False)[:10]

,CITY,ENR_TOTAL
511,Los Angeles,320297
789,San Diego,180201
798,San Jose,153385
54,Bakersfield,118654
777,Sacramento,118613
318,Fresno,114321
886,Stockton,75829
505,Long Beach,68485
817,Santa Ana,67595
762,Riverside,62857


In [114]:
combined_enr_schools = pd.merge(publicschools_enr_cities, privateschools_enr_cities, how='outer', left_on="CITY", right_on="City")

def calculate_percentage_private(row):
    # return percentage private enrollment over total enrollment
    return row["Total Enrollment"] / (row["Total Enrollment"] + row["ENR_TOTAL"])

combined_enr_schools["private_pct"] = combined_enr_schools.apply(lambda x: calculate_percentage_private(x), axis=1)
combined_enr_schools["total_enr"] = combined_enr_schools.apply(lambda x: x["ENR_TOTAL"] + x["Total Enrollment"], axis=1)
combined_enr_schools.sort_values("total_enr", ascending=False)[30:45]

,CITY,ENR_TOTAL,City,Total Enrollment,private_pct,total_enr
964,Visalia,31365,Visalia,1530,0.046512,32895
856,Simi Valley,30481,Simi Valley,2170,0.066460,32651
278,Escondido,30762,Escondido,1618,0.049969,32380
825,Santa Maria,30016,Santa Maria,1586,0.050187,31602
634,Oceanside,30724,Oceanside,685,0.021809,31409
392,Huntington Beach,27388,Huntington Beach,3603,0.116260,30991
638,Ontario,28905,Ontario,1503,0.049428,30408
961,Victorville,29825,Victorville,478,0.015774,30303
260,El Cajon,27782,El Cajon,1730,0.058620,29512
670,Pasadena,20637,Pasadena,8158,0.283313,28795
